In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Sollen-R/DS300-Final/main/dataset_phishing.csv?token=GHSAT0AAAAAABZL7ET5L5JNUN7SKHB7RBSCYZ4QMJA')

In [3]:
df.head()

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 82 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   length_url                  11430 non-null  int64  
 1   length_hostname             11430 non-null  int64  
 2   ip                          11430 non-null  int64  
 3   nb_dots                     11430 non-null  int64  
 4   nb_hyphens                  11430 non-null  int64  
 5   nb_at                       11430 non-null  int64  
 6   nb_qm                       11430 non-null  int64  
 7   nb_and                      11430 non-null  int64  
 8   nb_eq                       11430 non-null  int64  
 9   nb_underscore               11430 non-null  int64  
 10  nb_tilde                    11430 non-null  int64  
 11  nb_percent                  11430 non-null  int64  
 12  nb_slash                    11430 non-null  int64  
 13  nb_star                     114

In [8]:
for val in list(df.isna().sum()):
    if val != 0:
        print(val + ' Missing Values')
print('Done')

Done


In [17]:
df.drop('url', axis='columns', inplace=True)

In [23]:
useless_cols = []

for col in df:
    if len(df[col].value_counts()) == 1:
        useless_cols.append(col)
        
print(useless_cols)

['nb_or', 'ratio_nullHyperlinks', 'ratio_intRedirection', 'ratio_intErrors', 'submit_email', 'sfh']


In [24]:
df.drop(useless_cols, axis='columns', inplace=True)

In [31]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
ss = StandardScaler()
le = LabelEncoder()

df['status'] = le.fit_transform(df['status'])
df_scaled = pd.DataFrame(ss.fit_transform(df), columns=df.columns)

x = df_scaled.drop('status', axis='columns')
y = df_scaled['status']

In [38]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.2, random_state=8675309)

In [39]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

dtc = DecisionTreeClassifier(max_depth=10, random_state=8675309)
rfc = RandomForestClassifier(n_estimators=200, max_depth=2, random_state=8675309)
ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, random_state=8675309), n_estimators=600, random_state=8675309)
lrc = LogisticRegression(random_state=8675309)
gnb = GaussianNB()

dtc.fit(x_train, y_train)
rfc.fit(x_train, y_train)
ada.fit(x_train, y_train)
lrc.fit(x_train, y_train)
gnb.fit(x_train, y_train)

GaussianNB()

In [40]:
from sklearn.metrics import accuracy_score

print('Decision Tree:', accuracy_score(y_test, dtc.predict(x_test)))
print('Random Forest:', accuracy_score(y_test, rfc.predict(x_test)))
print('AdaBoost:', accuracy_score(y_test, ada.predict(x_test)))
print('Logistic Regression:', accuracy_score(y_test, lrc.predict(x_test)))
print('Gaussian Naive Bayes:', accuracy_score(y_test, gnb.predict(x_test)))

Decision Tree: 0.9185258092738408
Random Forest: 0.9292432195975503
AdaBoost: 0.9383202099737533
Logistic Regression: 0.9396325459317585
Gaussian Naive Bayes: 0.6851487314085739
